## Perform traditional bulk pycytominer pipeline

Following single-cell curation with cytotable, we create bulk profiles by applying the following steps:

1. aggregation
2. annotation
3. normalization
4. feature_selection

In [1]:
import pathlib
import yaml
import pprint

import pandas as pd

from pycytominer import aggregate, annotate, normalize, feature_select
from pycytominer.cyto_utils import load_profiles, output

In [2]:
# Set constants
feature_select_ops = [
    "variance_threshold",
    "correlation_threshold",
    "blocklist",
    "drop_na_columns",
]

# Set paths
output_dir = pathlib.Path("data/bulk_profiles")
output_dir.mkdir(exist_ok=True)
metadata_dir = pathlib.Path("../0.download_data/metadata/")

# load in plate information
dictionary_path = pathlib.Path("./plate_info_dictionary.yaml")
with open(dictionary_path) as file:
    plate_info_dictionary = yaml.load(file, Loader=yaml.FullLoader)

In [3]:
# add path to platemaps for each plate
for plate in plate_info_dictionary.keys():
    # since Plate_3_prime has the same platemap as Plate_3,
    # we need an else statement so that we make sure it adds the
    # path that was given to Plate_3
    if plate != "Plate_3_prime":
        # match the naming format of the plates to the platemap file
        plate_info_dictionary[plate]["platemap_path"] = str(
            pathlib.Path(
                list(
                    metadata_dir.rglob(
                        f"platemap_NF1_{plate.replace('_', '').lower()}.csv"
                    )
                )[0]
            ).resolve(strict=True)
        )
    else:
        plate_info_dictionary["Plate_3_prime"]["platemap_path"] = plate_info_dictionary[
            "Plate_3"
        ]["platemap_path"]

# view the dictionary to assess that all info is added correctly
pprint.pprint(plate_info_dictionary, indent=4)

{   'Plate_3': {   'dest_path': 'data/converted_data/Plate_3.parquet',
                   'platemap_path': '/home/jenna/nf1_cellpainting_data/0.download_data/metadata/platemap_NF1_plate3.csv',
                   'source_path': '/home/jenna/nf1_cellpainting_data/2.cellprofiler_analysis/analysis_output/Plate_3/Plate_3_nf1_analysis.sqlite'},
    'Plate_3_prime': {   'dest_path': 'data/converted_data/Plate_3_prime.parquet',
                         'platemap_path': '/home/jenna/nf1_cellpainting_data/0.download_data/metadata/platemap_NF1_plate3.csv',
                         'source_path': '/home/jenna/nf1_cellpainting_data/2.cellprofiler_analysis/analysis_output/Plate_3_prime/Plate_3_prime_nf1_analysis.sqlite'},
    'Plate_4': {   'dest_path': 'data/converted_data/Plate_4.parquet',
                   'platemap_path': '/home/jenna/nf1_cellpainting_data/0.download_data/metadata/platemap_NF1_plate4.csv',
                   'source_path': '/home/jenna/nf1_cellpainting_data/2.cellprofiler_analy

## Perform pycytominer pipeline

In [4]:
for plate, info in plate_info_dictionary.items():
    print(f"Now performing pycytominer pipeline for {plate}")
    output_aggregated_file = str(pathlib.Path(f"{output_dir}/{plate}_bulk.parquet"))
    output_annotated_file = str(
        pathlib.Path(f"{output_dir}/{plate}_bulk_annotated.parquet")
    )
    output_normalized_file = str(
        pathlib.Path(f"{output_dir}/{plate}_bulk_normalized.parquet")
    )
    output_feature_select_file = str(
        pathlib.Path(f"{output_dir}/{plate}_bulk_feature_selected.parquet")
    )

    # Load single-cell profiles
    single_cell_df = pd.read_parquet(info["dest_path"])

    # Load platemap
    platemap_df = pd.read_csv(info["platemap_path"])

    # Step 1: Aggregation
    aggregate(
        population_df=single_cell_df,
        operation="median",
        strata=["Image_Metadata_Plate", "Image_Metadata_Well"],
        output_file=output_aggregated_file,
        output_type="parquet",
    )

    # Step 2: Annotation
    annotated_df = annotate(
        profiles=output_aggregated_file,
        platemap=platemap_df,
        join_on=["Metadata_well_position", "Image_Metadata_Well"],
    )

    # For only plates 3 and 3 prime, remove any rows with HET due to contamination
    if plate in ["Plate_3", "Plate_3_prime"]:
        # Filter single-cell profiles, removing HET genotype
        annotated_df = annotated_df[annotated_df["Metadata_genotype"] != "HET"]
        print("HET cells have been removed from", plate)

    # use output to save annotated df as you can not use the output parameter in the annotate which will not return data frame
    output(
        df=annotated_df,
        output_filename=output_annotated_file,
        output_type="parquet",
    )

    # Step 3: Normalization
    normalized_df = normalize(
        profiles=annotated_df,
        method="standardize",
        output_file=output_normalized_file,
        output_type="parquet",
    )

    # Step 4: Feature selection
    feature_select(
        output_normalized_file,
        operation=feature_select_ops,
        na_cutoff=0,
        output_file=output_feature_select_file,
        output_type="parquet",
    )

Now performing pycytominer pipeline for Plate_3
HET cells have been removed from Plate_3
Now performing pycytominer pipeline for Plate_3_prime
HET cells have been removed from Plate_3_prime
Now performing pycytominer pipeline for Plate_4
Now performing pycytominer pipeline for Plate_5


In [5]:
# Check output file
test_df = load_profiles(output_feature_select_file)

print(test_df.shape)
test_df.head(2)

(72, 1140)


,Metadata_WellRow,Metadata_WellCol,Metadata_gene_name,Metadata_genotype,Metadata_Plate,Metadata_Well,Cytoplasm_AreaShape_Compactness,Cytoplasm_AreaShape_Eccentricity,Cytoplasm_AreaShape_Extent,Cytoplasm_AreaShape_FormFactor,...,Nuclei_Texture_InfoMeas1_RFP_3_01_256,Nuclei_Texture_InfoMeas1_RFP_3_02_256,Nuclei_Texture_InfoMeas1_RFP_3_03_256,Nuclei_Texture_InfoMeas2_DAPI_3_03_256,Nuclei_Texture_InfoMeas2_GFP_3_00_256,Nuclei_Texture_InfoMeas2_RFP_3_02_256,Nuclei_Texture_InverseDifferenceMoment_RFP_3_00_256,Nuclei_Texture_InverseDifferenceMoment_RFP_3_01_256,Nuclei_Texture_InverseDifferenceMoment_RFP_3_02_256,Nuclei_Texture_InverseDifferenceMoment_RFP_3_03_256
0,B,1,NF1,WT,Plate_5,B1,0.563493,0.347947,-0.215954,-0.620699,...,-0.282006,0.596654,0.605529,0.179448,-0.276720,-0.726135,-1.308387,-0.020369,-1.094164,-0.360964
1,C,1,NF1,WT,Plate_5,C1,0.114288,1.002189,0.451169,-0.190552,...,0.462571,0.605548,0.150431,0.393493,-0.368706,-0.026111,1.020429,-0.706318,-0.065927,0.388344
